2012年1月1日以降のデータで平均と分散共分散行列を求めるには、以下のように`pd.read_csv()`関数で`ReturnsData.csv`ファイルを読み込み、`loc`属性を使って2012年1月1日以降のデータを抽出し、`mean()`メソッドと`cov()`メソッドを使って平均と分散共分散行列を求めることができます。



In [190]:
import pandas as pd
import numpy as np

# Read data from ReturnsData.csv
returns = pd.read_csv("/workspaces/GPRtest/MarketData/ReturnsData.csv")

# Convert 日付 column to datetime type
returns["日付"] = pd.to_datetime(returns["日付"])

# Set 日付 column as index
returns = returns.set_index("日付")

# Exclude data before 2012-01-01
start_date = "2000-01-01"
returns = returns.loc[start_date:]

# Calculate mean returns for each column (annualized)
mean_returns = returns.mean() * 252

mean_returns.to_csv(f"/workspaces/GPRtest/MarketData/mean_returns{start_date}.csv")

# Calculate covariance matrix for returns (annualized)
cov_matrix = returns.cov() * 252
cov_matrix.to_csv(f"/workspaces/GPRtest/MarketData/cov_matrix{start_date}.csv") 

# Calculate correlation matrix for returns
corr_matrix = returns.corr()
corr_matrix.to_csv(f"/workspaces/GPRtest/MarketData/corr_matrix{start_date}.csv")

# print("Mean returns (annualized):")
# print(mean_returns)

# print("Covariance matrix (annualized):")
# print(cov_matrix)

# print("Correlation matrix:")
# print(corr_matrix)




このようにすることで、`ReturnsData.csv`ファイルから2012年1月1日以降のデータを抽出し、年率の平均と分散共分散行列を求めることができます。

In [191]:
#　Read data from MarketCap.csv
market_cap = pd.read_csv("/workspaces/GPRtest/MarketData/MarketCap.csv")
market_cap_ratio = market_cap/float(market_cap.sum(axis=1))

/tmp/ipykernel_23380/1341591377.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  market_cap_ratio = market_cap/float(market_cap.sum(axis=1))


#### Markowitz解
$$
\mu_m = a \Sigma \omega_m
$$

In [192]:
a = 1.5 # risk aversion parameter
mu1 = a*(cov_matrix.values).dot((market_cap_ratio.values).T)   # mean returns (annualized)
mu2 = (mean_returns.values).reshape(-1,1)
print(type(mu2))
print(mu2.shape)
print(mu2)

<class 'numpy.ndarray'>
(7, 1)
[[0.03726031]
 [0.07797926]
 [0.02428742]
 [0.07208445]
 [0.0213689 ]
 [0.06692769]
 [0.02485173]]


### 最適ポートフォリオ：無リスク資産がない場合
補助変数の導入
$$
\begin{align*}
A &= \mathbb{1}' \Sigma^{-1} \mathbb{1}  \\
B &= \mu' \Sigma^{-1} \mathbb{1} \\
C &= \mu' \Sigma^{-1} \mu
\end{align*}
$$
未定定数の計算
$$
\begin{align*}
\lambda_1 &= \frac{A \mu_p - B}{AC - B^2} \\
\lambda_2 &= \frac{C - B \mu_p}{AC - B^2}
\end{align*}
$$
ポートフォリオの重みと標準偏差
$$
\begin{align*}
\omega_p &= \lambda_1 \Sigma^{-1} \mu + \lambda_2 \Sigma^{-1} \mathbb{1}  \\
\sigma_p &= \sqrt{\omega_p' \Sigma \omega_p}
\end{align*}
$$

In [193]:
mu = mu1
vo = np.ones((len(mu),1)) # vector of ones
Sig = cov_matrix.values # covariance matrix (annualized)
A = (np.dot(vo.T, np.linalg.inv(Sig))).dot(vo) # a scalar
B = (np.dot(mu.T, np.linalg.inv(Sig))).dot(vo) # a scalar
C = (np.dot(mu.T, np.linalg.inv(Sig))).dot(mu) # a scalar

# 未定乗数の計算
lambda1 = lambda mup: (A*mup-B)/(A*C-B**2)
lambda2 = lambda mup: (C-B*mup)/(A*C-B**2)

# ポートフォリオの重みの計算
w = lambda mup: lambda1(mup)*np.linalg.inv(Sig).dot(mu)+lambda2(mup)*np.linalg.inv(Sig).dot(vo)

# ポートフォリオの標準偏差の計算
sigp = lambda mup: np.sqrt(w(mup).T.dot(Sig).dot(w(mup)))

In [ ]:
# Create Array from 0.01 to 0.04 with a step size 0.001
mups = np.arange(0.01, 0.3, 0.001)

# Calculate portfolio weights and portfolio volatility for each mup
weights = []
volatility = []
for mup in mups:
    weights.append(w(mup).tolist())
    volatility.append(sigp(mup)[0])

print(weights[50])
print(len(weights[0]))
print(type(volatility[0]))
# mupsとvolatilityのグラフを作成

import matplotlib.pyplot as plt

plt.plot(volatility, mups)  # x軸をvolatility, y軸をmupsとしてプロット
plt.xlabel("volatility")
plt.ylabel("expected return")
plt.grid(True)
plt.show()

###  無リスク資産がある場合
$$
\begin{align*}
\lambda &= \frac{\mu_p - r_f}{(\mu - r_f \mathbb{1})' \Sigma^{-1} (\mu - r_f \mathbb{1})}    \\ 
\omega_p &= \lambda \Sigma^{-1} (\mu - r_f \mathbb{1}) \\
\sigma_p &= \sqrt{\omega_p' \Sigma \omega_p}
\end{align*}
$$

In [ ]:
rf = 0.01 # risk free rate
lambda1 = lambda mup: (mup-rf)/((mu-rf*vo).T.dot(np.linalg.inv(Sig)).dot(mu-rf*vo)) # 未定乗数の計算
wf = lambda mup: lambda1(mup)*np.linalg.inv(Sig).dot(mu-rf*vo) # ポートフォリオの重みの計算
sigfp = lambda mup: np.sqrt(wf(mup).T.dot(Sig).dot(wf(mup))) # ポートフォリオの標準偏差の計算

# Calculate portfolio weights and portfolio volatility for each mup
weightfs = []
volatilityfs = []
for mup in mups:
    weightfs.append(wf(mup).tolist())
    volatilityfs.append(sigfp(mup)[0])

# mupsとvolatilityfsのグラフを作成
import matplotlib.pyplot as plt
plt.plot(volatility, mups)  # x軸をvolatility, y軸をmupsとしてプロット
plt.plot(volatilityfs, mups)  # x軸をvolatilityfs, y軸をmupsとしてプロット
plt.xlabel("volatility")
plt.ylabel("expected return")
plt.grid(True)
plt.show()

### Black-Littermanモデル
$$
\begin{align*}
G &= (F' \Omega^{-1} F + (\tau \Sigma)^{-1})^{-1} \\
H &= F' \Omega^{-1} r_v + (\tau \Sigma)^{-1} \mu 
\end{align*}
$$
$F$はviewを表す行列,$r_v$はviewの期待値を表すベクトル,$\Omega$はviewの共分散行列を表す行列,$\tau$は事前分布の分散・共分散に対する調整パラメータです。
事後分布の共分散は$G$で、事後分布の平均は$GH$で表されます。